# Import

In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns

# Note: The following do not work with Python 3.12
import shap
from ydata_profiling import ProfileReport
import sweetviz as sv

In [ ]:
file1 = '../Data/diet.csv'
file2 = '../Data/recipes.csv'
file3 = '../Data/requests.csv'
file4 = '../Data/reviews.csv'

diet = pd.read_csv(file1)
recipes = pd.read_csv(file2)
requests = pd.read_csv(file3)
reviews = pd.read_csv(file4)

# Reviews
Reviews has very important information about the User's opnion about a center Recipe.

In [ ]:
reviews.info()

In [ ]:
reviews['Rating'].unique()
reviews['Like'].values

__Rating__: is either 2.0 or nan, which isn't very informative, we can get rid of it.

__TestSetId__: is something we can't use, so we can get rid of it.

__Like__: has nan values, these rows won't provide any information for us, so we can delete these rows. 

In [ ]:
reviews = reviews.drop(columns=['Rating','TestSetId' ],errors='ignore')
reviews = reviews.dropna()
reviews['Like'] = reviews['Like'].astype('category')
reviews.info()

# Diet

In [ ]:
diet.info()

In [ ]:
diet['Diet'].unique()

__Age__: is very likely to be non-informative to our tastk

__Diet__: is perhaps useful, we now make it as categorical

In [ ]:
diet = diet.drop(columns=['Age' ],errors='ignore')
diet = diet.dropna()
diet['Diet'] = diet['Diet'].astype('category')

# Recipes

In [ ]:
recipes.head()

In [ ]:
recipes.info()

### First Glance

__Name__: very difficult to deal with, we can only use it with assign word with token about if it's vegan. But it's too much work, let's drop it for now.

__RecipeCategory__: array(['Other', 'Beverages', 'Bread', 'One dish meal', 'Breakfast', 'Lunch', 'Soup'], dtype=object) not much use 

__RecipeIngredientQuantities__: no use

__RecipeIngredientParts__: no use

__RecipeYield__: no use

__RecipeServings__: no use

In [ ]:
recipes = recipes.drop(columns=['Name','RecipeServings','RecipeCategory','RecipeIngredientQuantities','RecipeIngredientParts','RecipeYield'],errors='ignore')

### Second Glance

__CookTime__ && __PrepTime__: they should added into one number for total time, that's the number request want

__ProteinContent__: IMPORTANT * in requests

__FatContent__: IMPORTANT * in requests

__SugarContent__: IMPORTANT * in requests

__FiberContent__: IMPORTANT * in requests

__CarbohydrateContent__: not used in requests, high covariance with SugarContent FiberContent Calories, drop it for now

__SaturatedFatContent__: not used in requests, but high covariance with FatContent, drop it for now

__SodiumContent__: not used in requests, also have low covariances with other nuitrition, drop it

__CholesterolContent__: not used in requests, very high covariance many other Content, drop it for now

In [ ]:
recipes['Time'] = recipes['CookTime'] + recipes['PrepTime']
recipes = recipes.drop(columns=['CookTime','PrepTime'],errors='ignore')

recipes = recipes.drop(columns=['CarbohydrateContent','SaturatedFatContent','SodiumContent', 'CholesterolContent'],errors='ignore')

In [ ]:
# We can also leverage the dataprep package to get a nice summary report

report = sv.analyze(recipes)
report.show_html(filepath='recipes1.html')

# We can also leverage the yadata_profiling package to get a nice summary report
profile = ProfileReport(recipes, title="Recipes Review")
profile.to_file("recipes2.html")

# Requests

In [ ]:
requests.info()

In [ ]:
requests

All information are useful, just need to do some wrangling.

In [ ]:
requests['Time'] = requests['Time'].astype(int)

requests['HighCalories'] = requests['HighCalories'].map({0. : False, 1. : True})
requests['HighProtein'] = requests['HighProtein'].map({'Indifferent' : False, 'Yes' : True})
requests['LowFat'] = requests['LowFat'].map({0 : False, 1 : True})
requests['LowSugar'] = requests['LowSugar'].map({'0' : False, 'Indifferent' : True})
requests['HighFiber'] = requests['HighFiber'].map({0 : False, 1 : True})

requests = requests.rename(columns={'HighProtein' : 'HighProOrNotCare', 'LowSugar':'LowSugarOrDontCare'})